### Инструкция по выполнению

    1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).
    Проведите предобработку:

    Приведите тексты к нижнему регистру (text.lower()).
    Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:
    train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

    Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
    Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
    Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
    Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

    3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

    4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков.

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.

In [20]:
import pandas as pd
from pathlib import Path
path = Path.cwd()
path = path.joinpath('../data/raw/HSE_ML_week4')

train = pd.read_csv(path.joinpath('salary-train.csv'))
train.head(10)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
5,Sales and Marketing Assistant will provide adm...,Crawley,NaN,22500
6,Vacancy Ladieswear fashion Area Manager / Regi...,UK,permanent,32000
7,Reference: LR/JAN/**** Our client is one of th...,Bristol,permanent,30000
8,Sponsorship Manager London The Company A marke...,Central London,permanent,31500
9,"About Barclays Barclays moves, lends, invests ...",South East London,permanent,42499


In [28]:
# Handling misses
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

# Delete excessive symbols
train['FullDescription'] = train['FullDescription'].replace(
    '[^a-zA-Z0-9]', ' ', regex=True
)

# Lower texts
train['FullDescription'] = train['FullDescription'].transform(
    lambda text: text.lower()
)
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

vectorizer = TfidfVectorizer(min_df=5)
X_train_vec = vectorizer.fit_transform(train['FullDescription'])

enc = DictVectorizer()
X_train_categ = enc.fit_transform(
    train[['LocationNormalized', 'ContractTime']].to_dict('records')
)

In [33]:
from sklearn.linear_model import Ridge

y_train = train['SalaryNormalized']
X_train = hstack([X_train_vec, X_train_categ])

clf = Ridge(random_state=241)
clf.fit(X_train, y_train)

Ridge(random_state=241)

In [34]:
test = pd.read_csv(path.joinpath('salary-test-mini.csv'))
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [36]:
# Delete excessive symbols
test['FullDescription'] = test['FullDescription'].replace(
    '[^a-zA-Z0-9]', ' ', regex=True
)

# Lower texts
test['FullDescription'] = test['FullDescription'].transform(
    lambda text: text.lower()
)
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [38]:
X_test_vec = vectorizer.transform(test['FullDescription'])
X_test_categ = enc.transform(
    test[['LocationNormalized', 'ContractTime']].to_dict('records')
)

X_test = hstack([X_test_vec, X_test_categ])
clf.predict(X_test)

array([56581.31152328, 37133.54332311])

In [39]:
file = open('/home/topcoder2k/HSE_ML/HSE_ML_week4_answers/predicted_salary.txt', 'w')
file.write('56581.31 37133.54')
file.close()